In [1]:
# In this notebook I do two examples of how to implement genetic algorithms
# from scratch

# Ref:  https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/


In [2]:
import numpy as np

In [3]:
def selection(pop,n_pop,scores, k = 3):

  #Select first parent
  selection_ix = np.random.randint(n_pop)
  # Randomly pick k-1 parents
  for ix in np.random.randint(0,n_pop,k-1):

    # Compare score of parents and select best score
    if scores[ix] < scores[selection_ix]:

      selection_ix = ix

  return pop[selection_ix]

In [4]:
def crossover(p1,p2,r_cross):
  # Given two parents
  # Copy parents
  c1,c2 = p1.copy(),p2.copy()
  # If crossover occures
  if np.random.rand()<r_cross:
    # Select a partition point
    pt = np.random.randint(1,len(p1)-2)
    # Crossover the genes
    c1 = p1[:pt]+p2[pt:]
    c2 = p2[:pt]+p1[pt:]

  return [c1,c2]

In [5]:
p1 = [0, 1, 0, 1, 0, 1, 1]
p2 =  [1, 0, 1, 0, 0, 1, 1]

pt = 4

c1 = p1[:pt] + p2[pt:]

c2 = p2[:pt] + p1[pt:]

print(c1)

print(c2)


[0, 1, 0, 1, 0, 1, 1]
[1, 0, 1, 0, 0, 1, 1]


In [6]:
def mutation(bitstring,n_bits,r_mut):
  # Given a bitstring
  # Go through each elemnt of the string
  # print(bitstring)
  for i in range(n_bits):
    # If mutation occurs
    if np.random.rand()<r_mut:
      # Flip the bit
      bitstring[i] = 1-bitstring[i]

  return bitstring



In [7]:
# First stpe create a population

In [8]:
# genetic algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross,r_mut):
  # initial population of random bitstring
  pop = [np.random.randint(0,2,n_bits).tolist() for _ in range(n_pop)]
  best, best_eval = 0,objective(pop[0])
  # enumerate generations
  for gen in range(n_iter):

    # print(gen)

    #evaluate all cadidates in the population

    scores = [objective(c) for c in pop]    

    # chec for new best solution

    for i in range(n_pop):
      if scores[i]< best_eval:
        best,best_eval = pop[i],scores[i]
        print(">%d, new best f(%s) = %.3f"%(gen,pop[i],scores[i]))
    # select parents 
    selected= [selection(pop,n_pop,scores) for _ in range(n_pop)]
    children = list()

    for i in range(0,n_pop,2):
      # Select parents in pairs
      p1, p2 = selected[i], selected[i+1]
      # print("p1")
      # print(p1)
      # print("p2")
      # print(p2)
      # Crossover and mutation 
      for c in crossover(p1,p2,r_cross):
        # Mutation
        #print("mut")
        mutation(c,n_bits,r_mut)
        # Store for next generation
        children.append(c)
      # Replace popuation
      pop = children
  return [best,best_eval]

We start with a simple example, the OneMax problem. It valuates a binary string based on the number of 1s in the string. The objective function in this case is the sum of 1s in a bitstring

In [9]:
# objective funciton 

def onemax(x):

  return -sum(x)

In [10]:
# define the total iterations 
n_iter = 100
# bits
n_bits = 20
# Pop size
n_pop = 100
# Crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0/float(n_bits)

In [11]:
best, score = genetic_algorithm(onemax, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
print('f(%s) = %f' % (best, score))

>2, new best f([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]) = -18.000
>4, new best f([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -19.000
>7, new best f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000
Done!
f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000000


In [12]:
import networkx as nx

In [136]:
G = nx.Graph()
G.add_edge(1,2)
G.add_edge(1,3)
G.add_edge(2,4)
G.add_edge(2,5)
G.add_edge(3,4)
G.add_edge(3,5)
G.add_edge(1,8)
G.add_edge(8,3)
G.add_edge(7,4)
G.add_edge(7,6)
G.add_edge(6,2)
G.add_edge(6,5)

In [137]:
num_edges_G =len(G.edges)

In [15]:
C = [2,3]

In [16]:
def num_edge_cover(C: list):
  # Function to obtain number of edges covered by C
  # Input: C -> Vertex subset 
  # Output: count -> int
  count = 0

  # For every vertex  u in C
  for u in C:
    # Count every conection v
    for v in list(G.adj[u]):
      # Avoid repeting edge (u,v) and (v,u)
      if v in C and v<u:
        continue
      else:
        # Add new edges 
        count +=1
  return count

In [40]:
def cost_cover(num_edges_G: int,C: list):
  # Function to asses the fitness of C
  # For a vertex cover returns 0 
  # Input:
  # num_edges_G: Number of edges in graph G
  # C: Strings of 1 and 0
  # Output:
  # Difference between number of edges and edges covered by C.


  #Obtain subset from binary string
  V = np.where(np.asanyarray(C)==1)[0]+1

  return num_edges_G-num_edge_cover(V)


In [18]:
cost_cover(num_edges_G , C)

0

We take the decision form of the problem. For a given $k$, is there a subset cover $C\subset V$ with $|C| = k$?

Since we will only be working with subsets of size $k$ every element of the population only has $k$ entries with $1$.

To define the genetic algorithm we require a crossover function and a mutation function. To remain in the correct population, we construct a corssover funciton and a mutation function that keep $k$  entries with value $1$ in any individual.

Idea for crossover: 
- Sample $p$ 1s indexes from $p_1,p_2$ lets say $a_1,a_2$
- For every element in $a_1$ check if the corresponding entry in $p_2$ is a $0.$ Save such valid elements of $a_1.$ Similarly for $a_2.$
- Let $m = \min(a_1,a_2).$
- Randomly sample $m$ elements from the valid lists and only change those locations for $c_1,c_2$.

Idea for mutation:

- Given a parameter $n_mut$ which is the number of entries to mutate
- If $n_{mut} <k $ and $n_{mut}<n-k,$ find $n_{mut}$ entries with 1s and 0s. Exchange those locations to 0s and 1s, respectively.

This ensures that the amount of 1s in the mutated 

In [72]:
def crossover_min_vertex_cov(p1: list,p2: list,n_cross: int,r_cross: float):
  # Function for doing crossover of partents
  #Input:
  # p1,p2: parents
  # p: integer smaller than current vertex cover decision problem
  # r_cross: float for deciding if crossover occurs
  # Output:
  # c1,c2: lists. Childs of p1 and p2
  # Given two parents
  # Copy parents
  c1,c2 = p1.copy(),p2.copy()
  # If crossover occures
  if np.random.rand()<r_cross:
    # Select a partition point
    # replace = False. Avoid sampling one number more than once
    idx1 = np.random.choice(np.where(np.asarray(p1)==1)[0],n_cross,replace = False)
    idx2 = np.random.choice(np.where(np.asarray(p2)==1)[0],n_cross,replace = False)
    # print(idx1,idx2)
    
    idx1_valid = []
    idx2_valid = []
    # Find subsets of indexes to ensure that there are k 1s in each child
    # Only do a swap when for a 1 corresponds a 0
    for i in range(n_cross):

      if p2[idx1[i]] == 0:

        idx1_valid.append(idx1[i])   

      if p1[idx2[i]] ==0:

        idx2_valid.append(idx2[i])
    
    # Only swap shared amount of indexes correspondances

    m = min(len(idx1_valid),len(idx2_valid))

    # print(m,idx1_valid,idx2_valid)

    # Randomly select which indexes to change

    idx1 = np.random.choice(idx1_valid,m,replace = False)

    idx2 = np.random.choice(idx2_valid,m,replace = False)

    # print(idx1,idx2)

    # Change the indexes

    for i in range(m):

      c1[idx2[i]] = p2[idx2[i]]

      c1[idx1[i]] = 0

      c2[idx1[i]] = p1[idx1[i]]

      c2[idx2[i]] = 0

  #print("ch 1 %s,"%c1)
  #print("ch2 %s"%c2)

  return [c1,c2]

In [73]:
p1 = [0, 1, 0, 1, 1, 0, 1]
p2 =  [1, 0, 1, 0, 0, 1, 1]

In [74]:
crossover_min_vertex_cov(p1,p2,2,1)

[[1, 0, 1, 1, 0, 0, 1], [0, 1, 0, 0, 1, 1, 1]]

In [75]:
p1

[0, 1, 0, 1, 1, 0, 1]

In [76]:
p2

[1, 0, 1, 0, 0, 1, 1]

In [127]:
def mutation_min_vertex_cov(bitstring: list,n_bits: int,n_mut: int,r_mut: float, k :int):
  # Given a bitstring
  # Go through each elemnt of the string
  # print(bitstring)

  # bitstring_copy = bitstring.copy()

  if np.random.rand() < r_mut:
    if n_mut< k and n_mut< n_bits-k:

      idx1 = np.random.choice(np.where(np.asarray(bitstring)==1)[0],n_mut,replace = False)

      idx0 = np.random.choice(np.where(np.asarray(bitstring)==0)[0],n_mut,replace = False)

      # print(idx0,idx1)
    
      for i in range(n_mut):

        bitstring[idx0[i]] = 1

        bitstring[idx1[i]] = 0

  return bitstring


In [128]:
mutation_min_vertex_cov(p1.copy(),7,1,1,4)

[0, 0, 1, 1, 1, 0, 1]

In [129]:
p1

[0, 1, 0, 1, 1, 0, 1]

In [147]:
# genetic algorithm
def genetic_algorithm_min_vertex_decision(objective, n_bits, n_iter, n_pop, r_cross,r_mut,n_cross,n_mut,k,num_edges_G ):
  # initial population of random bitstring
  # pop = [np.random.randint(0,2,n_bits).tolist() for _ in range(n_pop)]
  pop = []
  for _ in range(n_pop):

    pop_elem = np.zeros(n_bits,dtype = int)
    for i in np.random.choice(n_bits,k,replace = False):
      pop_elem[i] = 1

    pop.append(pop_elem.tolist())

  # print(pop)

  best, best_eval = 0,objective(num_edges_G,pop[0])
  # enumerate generations
  for gen in range(n_iter):

    # print(gen)

    #evaluate all cadidates in the population

    scores = [objective(num_edges_G,c) for c in pop]    

    print(scores)

    # chec for new best solution

    for i in range(n_pop):
      if scores[i]< best_eval:
        best,best_eval = pop[i],scores[i]
        print(">%d, new best f(%s) = %.3f"%(gen,pop[i],scores[i]))

      if best_eval == 0:

        return [best,best_eval]

    # select parents 
    selected= [selection(pop,n_pop,scores) for _ in range(n_pop)]
    children = list()

    for i in range(0,n_pop,2):
      # Select parents in pairs
      p1, p2 = selected[i], selected[i+1]
      # print("p1")
      # print(p1)
      # print("p2")
      # print(p2)
      # Crossover and mutation 
      for c in crossover_min_vertex_cov(p1,p2,n_cross,r_cross):
        # Mutation
        #print("mut")
        mutation_min_vertex_cov(c,n_bits,n_mut,r_mut,k)
        # Store for next generation
        children.append(c)
      # Replace popuation
      pop = children
      # print(pop)
  return [best,best_eval]

In [178]:
# define the total iterations 
n_iter = 150
# bits
n_bits = len(G.nodes)
# Pop size
n_pop = 2
# Crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0/float(n_bits)
# decision problem 
k = 5
# Entries to crossover < k
n_cross = int(k/2)
# Entries to mutate <k and n_bits-k
n_mut = 1

In [179]:
genetic_algorithm_min_vertex_decision(cost_cover,n_bits,n_iter,n_pop,r_cross,r_mut,n_cross,n_mut,k,num_edges_G)

[1, 3]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[2, 2]
[1, 1]
[1, 2]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 2]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 2]
[2, 1]
[1, 2]
[2, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 2]
[1, 2]
[1, 2]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[2, 2]
[2, 2]
[1, 2]
[1, 2]
[2, 2]
[2, 1]
[1, 1]
[0, 1]
>55, new best f([0, 1, 1, 0, 0, 1, 1, 1]) = 0.000


[[0, 1, 1, 0, 0, 1, 1, 1], 0]

In [68]:
pop_elem

array([1, 1, 0, 1, 1])